# NEGATIVE SELECTION ALGORITHM(NSA) FOR SPAM DETECTION

## Imports

In [8]:
import string
import nltk

nltk.download('stopwords')

stop_words = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/fbb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Start with the data extraction

In [2]:
def data_extraction(file_path="Data/SMSSpamCollection"):
    """
    Given a filepath, extract the data within the file
    """
    try:
        with open(file_path, 'r') as f:
            data = [line.strip() for line in f]
            return data
    except FileNotFoundError:
        print(f"ERROR: File not found at {filepath}")
        return []

In [10]:
def data_processing(data, stop_words):
    """
    Given the data, return tuple of data and label
    """
    finished_data =[]
    for sms in data:
        label, text = sms.split("\t")
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
        words = text.split()
        words = [word for word in words if word not in stop_words]
        text = " ".join(words)
        finished_data.append((label, text))
    return finished_data

In [11]:
file_path = "Data/SMSSpamCollection"

data = data_extraction(file_path)
print(data[0])
print(data[1])

data = data_processing(data, stop_words)
print(data[0])
print(data[1])

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
('ham', 'go jurong point crazy available bugis n great world la e buffet cine got amore wat')
('ham', 'ok lar joking wif u oni')


In [12]:
max_str_len = 0
for string in data:
    str_len = len(string[1])
    if str_len > max_str_len:
        max_str_len = str_len
print(max_str_len)

516


## The NSA Class

The basic idea is to create a detector based only on the self, so the strings which are not harmful. So the detector is trained on random strings, which are matched against the 'ham' labeled data. 